## Stock Market Analysis

In this notebook, we will discover and explore data from the stock market, particularly some dead stocks. We will see how to use UpstoxApi by [Upstox](https://upstox.com/) to get stock historical data, for 1 min chart(open, high, low, close) and export it into CSV format

### Getting the Instrument details

* instrument_key	-->	The unique identifier used across Upstox APIs for instrument identification.
* exchange_token	-->	The numerical identifier issued by the exchange representing the instrument.
* tradingsymbol	    -->	Shows the trading symbol which could be a combination of symbol name, instrument, expiry date etc.
* name	            -->	Name of the company (for equity instruments).
* last_price	    -->	Last traded price.
* expiry	        -->	Expiry date (for derivatives). Data format is yyyy-MM-dd
* strike	        -->	Indicates the predetermined price at which an option can be bought or sold when it's exercised.
* tick_size	        -->	Measure of the minimum upward or downward movement in the price of an instrument.
* lot_size	        -->	Minimum size in which the stock futures or index futures can be traded.
* instrument_type	-->	Instrument type of a particular contract. Possible values: FUTSTK, OPTSTK, OPTIDX etc.
* option_type	    -->	Option type of the option contracts (applicable only for options contract). Possible values: CE, PE
* exchange	        -->	Exchange to which the order is associated.Possible values: NSE_EQ, NSE_FO, NCD_FO, NSE_INDEX, BSE_EQ, BSE_FO, BCD_FO, BSE_INDEX, MCX_FO, NSE_COM etc.

In [1]:
import gzip
import json

# Path to the compressed JSON file
file_path = "NSE.json.gz"

# Function to extract and read JSON data
def extract_json_data(file_path):
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Extract data
data = extract_json_data(file_path)

# Function to filter by segment
def filter_by_segment(data, segment=None, name=None, instrument_type=None, 
                      instrument_key=None, exchange_token=None, trading_symbol=None):
    
    filtered_data = []
    for item in data:
        if (
            (segment is None or item.get("segment") == segment)
            and (name is None or item.get("name") == name)
            and (instrument_type is None or item.get("instrument_type") == instrument_type)
            and (instrument_key is None or item.get("instrument_key") == instrument_key)
            and (exchange_token is None or item.get("exchange_token") == exchange_token)
            and (trading_symbol is None or item.get("trading_symbol") == trading_symbol)
        ):
            filtered_data.append(item)

    
    return filtered_data

# Example: Filter for 'NSE_EQ' segment
filteredTicker = filter_by_segment(data, "NSE_EQ", "INFOSYS LIMITED" , "EQ", None, None, None)

# Print the first 5 records from the filtered data
print(json.dumps(filteredTicker, indent=3))

instrument_key = filteredTicker[0]["instrument_key"]
print(instrument_key)

[
   {
      "segment": "NSE_EQ",
      "name": "INFOSYS LIMITED",
      "exchange": "NSE",
      "isin": "INE009A01021",
      "instrument_type": "EQ",
      "instrument_key": "NSE_EQ|INE009A01021",
      "lot_size": 1,
      "freeze_quantity": 100000.0,
      "exchange_token": "1594",
      "tick_size": 5.0,
      "trading_symbol": "INFY",
      "short_name": "Infosys",
      "qty_multiplier": 1.0,
      "security_type": "NORMAL"
   }
]
NSE_EQ|INE009A01021


### Importing Flask to set up our server

In [ ]:
import requests
import threading
from flask import Flask, request, jsonify, redirect, session

# Flask App
app = Flask(__name__)
app.secret_key = "2356"

ACCESS_TOKEN = None

@app.route('/')
def login():
    return redirect("https://api.upstox.com/v2/login/authorization/dialog?response_type=code&client_id=3146de73-0e14-4bac-af8c-b248c718e43e&redirect_uri=http://127.0.0.1:5000/callback&state=xyz")

@app.route('/callback')
def callback():
    global ACCESS_TOKEN

    code = request.args.get("code")
    if not code:
        return "Authorization failed", 400

    payload = {
        "code": code,
        "client_id": "3146de73-0e14-4bac-af8c-b248c718e43e",
        "client_secret": "tpfd7qwdmd",
        "redirect_uri": "http://127.0.0.1:5000/callback",
        "grant_type": "authorization_code",
    }
    response = requests.post("https://api.upstox.com/v2/login/authorization/token", data=payload)

    try:
        data = response.json()
        if "access_token" not in data:
            return jsonify({"error": data}), 400

        ACCESS_TOKEN = data["access_token"]  # Store in a global variable
        session["access_token"] = ACCESS_TOKEN
        return redirect("/profile")
    
    except requests.exceptions.JSONDecodeError:
        return "Failed to decode response from Upstox", 500

@app.route('/profile')
def profile():
    if "access_token" not in session:
        return redirect("/")
    
    headers = {"Authorization": f"Bearer {session['access_token']}"}
    response = requests.get("https://api.upstox.com/v2/user/profile", headers=headers)

    try:
        return jsonify(response.json())
    except requests.exceptions.JSONDecodeError:
        return "Failed to decode profile response", 500

# Run Flask in a background thread (if inside Jupyter)
def run_flask():
    app.run(debug=True, use_reloader=False)

threading.Thread(target=run_flask).start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Feb/2025 22:18:13] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [10/Feb/2025 22:18:40] "GET /callback?code=1UPS0w&state=xyz HTTP/1.1" 302 -
127.0.0.1 - - [10/Feb/2025 22:18:40] "GET /profile HTTP/1.1" 200 -
127.0.0.1 - - [10/Feb/2025 22:18:40] "GET /favicon.ico HTTP/1.1" 404 -


In [3]:
def make_request(method, url, header=None):
    response = None
    try:
        if method == 'GET':
            request = requests.get(url, headers=header)
        else:
            raise ValueError("Not Valid HTTP Method")
        
        if request.status_code == 200:
            return request.json()   
        else:
            return response
    except request.exceptions.RequestException as e:
        print(f'Error : {e}')
        return None

In [4]:
interval= "1minute"
from_date = "2025-02-07"
to_date = "2025-02-07"

url = f"https://api.upstox.com/v2/historical-candle/{instrument_key}/{interval}/{to_date}/{from_date}"

header = {
    'accept': 'application/json',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
    }

response = make_request('GET', url, header=header)  
print(response)

{'status': 'success', 'data': {'candles': [['2025-02-07T15:29:00+05:30', 1903.25, 1904.1, 1901, 1901.1, 57168, 0], ['2025-02-07T15:28:00+05:30', 1904.05, 1904.8, 1902.9, 1903.25, 38775, 0], ['2025-02-07T15:27:00+05:30', 1903.85, 1904.25, 1903.75, 1903.95, 26937, 0], ['2025-02-07T15:26:00+05:30', 1903.55, 1904.4, 1903.15, 1903.8, 27056, 0], ['2025-02-07T15:25:00+05:30', 1904.85, 1905, 1902.3, 1903.55, 40932, 0], ['2025-02-07T15:24:00+05:30', 1907.6, 1907.6, 1904.6, 1904.8, 44097, 0], ['2025-02-07T15:23:00+05:30', 1908.1, 1908.2, 1906.8, 1907.55, 35893, 0], ['2025-02-07T15:22:00+05:30', 1908.15, 1908.15, 1906.85, 1907.9, 31809, 0], ['2025-02-07T15:21:00+05:30', 1906.05, 1908.3, 1906, 1908.1, 29860, 0], ['2025-02-07T15:20:00+05:30', 1906.55, 1907, 1905.7, 1906, 35554, 0], ['2025-02-07T15:19:00+05:30', 1904.95, 1906.6, 1904.9, 1906.55, 33878, 0], ['2025-02-07T15:18:00+05:30', 1905.4, 1905.4, 1904.3, 1904.95, 33308, 0], ['2025-02-07T15:17:00+05:30', 1907.7, 1907.85, 1905, 1905, 50897, 0], [

In [5]:
import pandas as pd

# Extract candle data if response is valid
candles = response.get("data", {}).get("candles", [])

if candles:
    # Define column names dynamically
    required_columns = ["timestamp", "open", "high", "low", "close", "volume"]
    
    # Create DataFrame
    df = pd.DataFrame([candle[:6] for candle in candles], columns=required_columns)
    
    # Convert timestamp to datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"]).dt.strftime("%H:%M")
    
    # Display first 5 rows
    print(df.head())

  timestamp     open     high      low    close  volume
0     15:29  1903.25  1904.10  1901.00  1901.10   57168
1     15:28  1904.05  1904.80  1902.90  1903.25   38775
2     15:27  1903.85  1904.25  1903.75  1903.95   26937
3     15:26  1903.55  1904.40  1903.15  1903.80   27056
4     15:25  1904.85  1905.00  1902.30  1903.55   40932


In [6]:
df.to_csv("INFOSYS LIMITED.csv", index=False)